<a href="https://colab.research.google.com/github/NicholasSetiadi99/Churn_analysis_script/blob/main/Churn_Analysis_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 871.5 kB/s eta 0:00:00


In [4]:
import pymysql
import pandas as pd
pymysql.install_as_MySQLdb()
import MySQLdb
import numpy as np

# Set MySQL connection parameters
db_connection = MySQLdb.connect(host='1.1.1.1', database='mock_database', user='nicholas_test', password='xxxxx')
db_cursor = db_connection.cursor()

# Define months and column names
months = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
column_names = ["Month", "UserID", "MonthlyDeposit", "CumulativeDeposit", "MonthlyWithdrawal", "CumulativeWithdrawal", "fullWithdrawal?"]

# Create an empty list to store DataFrames
dfs = []

# SQL query template
query = '''
SELECT %s, u.email, IFNULL(monthlyDepAmt, 0), IFNULL(cumDepAmt, 0), IFNULL(monthlyWitAmt, 0), IFNULL(cumWitAmt, 0), IFNULL(fullWit, 0)
FROM (
    SELECT email, id, MONTH(date_created) AS "Month", YEAR(date_created) AS "Year"
    FROM USER
    WHERE MONTH(date_created) <= %s
    AND YEAR(date_created) = 2021
    AND kyc_status = "VERIFIED"
    AND status = "ACTIVE"
    AND id IN (SELECT user_id FROM DEPOSIT WHERE status IN ("APPROVED", "BOOKED"))
) u
LEFT JOIN (
    SELECT user_id, SUM(amount) AS monthlyDepAmt
    FROM DEPOSIT
    WHERE status IN ("APPROVED", "BOOKED")
    AND MONTH(date_created) = %s
    AND YEAR(date_created) = 2021
    GROUP BY user_id
) mD ON u.id = mD.user_id
LEFT JOIN (
    SELECT user_id, SUM(amount) AS monthlyWitAmt, MAX(is_full_withdrawal) AS fullWit
    FROM WITHDRAWAL
    WHERE status IN ("APPROVED", "BOOKED")
    AND MONTH(date_created) = %s
    AND YEAR(date_created) = 2021
    GROUP BY user_id
) mW ON u.id = mW.user_id
LEFT JOIN (
    SELECT user_id, SUM(amount) AS cumDepAmt
    FROM DEPOSIT
    WHERE status IN ("APPROVED", "BOOKED")
    AND MONTH(date_created) <= %s
    AND YEAR(date_created) = 2021
    GROUP BY user_id
) cD ON u.id = cD.user_id
LEFT JOIN (
    SELECT user_id, SUM(amount) AS cumWitAmt
    FROM WITHDRAWAL
    WHERE status IN ("APPROVED", "BOOKED")
    AND MONTH(date_created) <= %s
    AND YEAR(date_created) = 2021
    GROUP BY user_id
) cW ON u.id = cW.user_id;
'''

# Loop through months and execute the query
for month in months:
    db_cursor.execute(query, (month, month, month, month, month, month))
    table_rows = db_cursor.fetchall()
    df = pd.DataFrame(table_rows, columns=column_names)
    dfs.append(df)

# Concatenate the DataFrames
df = pd.concat(dfs, ignore_index=True)

# Sort the DataFrame
df = df.sort_values(by=['UserID', 'Month'], ascending=True)

# Calculate Churn and Comeback
df['Churn'] = np.where((df['CumulativeWithdrawal'] == 0) | (df['CumulativeWithdrawal'] < df['CumulativeDeposit']), 'No', 'Churn')
df['Comeback'] = np.where((df['UserID'].shift() == df['UserID']) & (df['CumulativeDeposit'] > df['CumulativeWithdrawal']) & (df['Churn'].shift() == 'Churn'), 'Comeback', 'No')

# Export the DataFrame to a CSV file
df.to_csv('churn_analysis_script2021.csv', index=False)

# Close the MySQL connection
db_cursor.close()
db_connection.close()

OperationalError: ignored